In [1]:
import librosa
import os
import pandas as pd
import numpy as np

In [4]:
#current_path = os.path.dirname(os.path.realpath(__file__))

songs = pd.read_csv("C:/Users/lly/Desktop/CUHK/Year 4/Sem 1/AIST2010/Project/Singers/dataset.csv")

audio_files_dir = ("C:/Users/lly/Desktop/CUHK/Year 4/Sem 1/AIST2010/Project/Singers/dataset")
audio_files = os.listdir(audio_files_dir)

In [32]:
# 100 audio files
audio_data = []
for file in audio_files:
    file_path = os.path.join(audio_files_dir, file)
    data, sr = librosa.load(file_path, sr=22050)
    data = librosa.to_mono(data)
    data = librosa.util.normalize(data)

    #num_segments = len(data) // sr
    segments = np.array_split(data[:25 * sr], 25)

    audio_data.append(segments)

In [33]:
final_dataset = []
for i in audio_data:
    final_dataset.extend(i)

In [34]:
final_dataset = np.array(final_dataset)

In [35]:
final_dataset.shape

(2500, 22050)

In [40]:
def mfcc(data):
    # Mel-frequency cepstral coefficients (MFCCs)
    mfcc_audio = librosa.feature.mfcc(y=data, sr=22050, n_fft=512)
    mfcc_audio = mfcc_audio.flatten()
    return mfcc_audio

In [41]:
mfcc_audio_data = []
for segment in final_dataset:
    mfcc_audio = mfcc(segment)
    mfcc_audio_data.append(mfcc_audio)

In [42]:
mfcc_audio_data = np.array(mfcc_audio_data)

In [43]:
mfcc_audio_data.shape

(2500, 880)

In [44]:
mfcc_audio_data

array([[-197.73824  , -245.47433  , -244.586    , ...,   19.019123 ,
          28.69128  ,   18.582964 ],
       [-263.1695   , -346.9509   , -387.94293  , ...,  -11.384357 ,
         -13.385475 ,   -6.5554695],
       [-226.48969  , -251.4339   , -254.16122  , ...,  -19.222832 ,
         -23.233484 ,   -3.8466933],
       ...,
       [-242.54521  , -254.78162  , -243.12556  , ...,   16.782604 ,
          28.263494 ,   13.888154 ],
       [-273.47614  , -323.56808  , -333.13507  , ...,   17.449774 ,
          14.415916 ,   17.628853 ],
       [-242.38847  , -274.5683   , -260.1299   , ...,   -3.8547637,
           8.730096 ,    7.020806 ]], dtype=float32)

In [45]:
from sklearn.preprocessing import StandardScaler

In [47]:
scaler = StandardScaler()
mfcc_audio_data = scaler.fit_transform(mfcc_audio_data)

In [48]:
mfcc_audio_data

array([[ 0.7232258 ,  0.635     ,  0.64371514, ...,  0.73178315,
         1.2846894 ,  1.5907325 ],
       [ 0.19124846, -0.26539257, -0.64612335, ..., -0.9354239 ,
        -1.0571544 , -0.9118112 ],
       [ 0.4894672 ,  0.5821213 ,  0.5575631 , ..., -1.365255  ,
        -1.6052599 , -0.6421512 ],
       ...,
       [ 0.35893056,  0.55241734,  0.6568552 , ...,  0.6091413 ,
         1.2608804 ,  1.1233618 ],
       [ 0.10745215, -0.05791895, -0.15299551, ...,  0.64572626,
         0.49017298,  1.4957502 ],
       [ 0.3602049 ,  0.37685183,  0.5038605 , ..., -0.5225307 ,
         0.17372029,  0.43971384]], dtype=float32)

In [51]:
final_dataset_df = pd.read_csv("C:/Users/lly/Desktop/CUHK/Year 4/Sem 1/AIST2010/Project/Singers/final_dataset_df.csv")

In [52]:
Y_label = np.array(final_dataset_df[final_dataset_df.columns[-1]])

In [54]:
from sklearn.preprocessing import LabelEncoder

In [55]:
label_encoder = LabelEncoder()
Y_label_encoded = label_encoder.fit_transform(Y_label)

In [56]:
import torch

In [57]:
X_tensor = torch.tensor(mfcc_audio_data, dtype=torch.float32)
Y_label_encoded_tensor = torch.tensor(Y_label_encoded, dtype=torch.long)

In [58]:
from sklearn.model_selection import train_test_split

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X_tensor, Y_label_encoded_tensor, test_size=0.2, random_state=42)

In [60]:
X_train = X_train.unsqueeze(1)
X_test = X_test.unsqueeze(1)

In [61]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import torch.nn.functional as F
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [62]:
class CNNModel(nn.Module):
    def __init__(self, input_size, num_classes):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * (input_size // 16), 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        x = self.fc2(x)
        return x

In [63]:
num_classes = len(label_encoder.classes_)
input_size = mfcc_audio_data.shape[1]
model = CNNModel(input_size, num_classes)

In [64]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [65]:
num_epochs = 70
for epoch in tqdm(range(num_epochs)):
    outputs = model(X_train)
    outputs = outputs.squeeze(1)
    loss = criterion(outputs, y_train)

    _, predicted_labels = torch.max(outputs, 1)
    accuracy = accuracy_score(y_train.numpy().flatten(), predicted_labels.numpy())

    '''model.eval()
    test_outputs = model(X_test)
    test_outputs = test_outputs.squeeze(1)
    y_test = y_test.view(-1)

    test_loss = criterion(test_outputs, y_test)

    _, predicted_test_labels = torch.max(test_outputs, 1)
    test_accuracy = accuracy_score(y_test.numpy().flatten(), predicted_test_labels.numpy())

    print(f'Test Loss: {test_loss.item():.4f}, Test Accuracy: {test_accuracy:.2%}')'''


    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f"Epoch [{epoch + 1}/{num_epochs}], CNN Loss: {loss.item():.4f}, CNN Accuracy: {accuracy:.2%}")

print("Training loop ended...")

  1%|▏         | 1/70 [00:03<04:07,  3.58s/it]

Epoch [1/70], CNN Loss: 2.3033, CNN Accuracy: 9.45%


  3%|▎         | 2/70 [00:07<04:30,  3.98s/it]

Epoch [2/70], CNN Loss: 24.7642, CNN Accuracy: 10.70%


  4%|▍         | 3/70 [00:11<04:24,  3.94s/it]

...

Epoch [68/70], CNN Loss: 0.0849, CNN Accuracy: 96.80%


 99%|█████████▊| 69/70 [05:52<00:06,  6.20s/it]

Epoch [69/70], CNN Loss: 0.0871, CNN Accuracy: 97.40%


100%|██████████| 70/70 [05:58<00:00,  5.12s/it]

Epoch [70/70], CNN Loss: 0.1045, CNN Accuracy: 96.50%
Training loop ended...


In [66]:
with torch.no_grad():
    model.eval()
    test_outputs = model(X_test)
    test_outputs = test_outputs.squeeze(1)
    y_test = y_test.view(-1)

    test_loss = criterion(test_outputs, y_test)

    _, predicted_test_labels = torch.max(test_outputs, 1)
    test_accuracy = accuracy_score(y_test.numpy().flatten(), predicted_test_labels.numpy())

    print(f'Test Loss: {test_loss.item():.4f}, Test Accuracy: {test_accuracy:.2%}')

Test Loss: 1.8775, Test Accuracy: 67.20%


In [67]:
print("Model Evaluation Completed!")
print(y_test)
print(predicted_test_labels)

Model Evaluation Completed!
tensor([5, 4, 4, 9, 6, 2, 6, 9, 1, 9, 1, 0, 3, 4, 2, 3, 5, 6, 1, 4, 8, 2, 8, 9,
        0, 3, 5, 4, 5, 2, 5, 0, 4, 8, 3, 4, 7, 5, 7, 5, 9, 5, 7, 5, 3, 6, 5, 0,
        7, 9, 1, 7, 2, 0, 0, 0, 4, 5, 7, 5, 9, 5, 1, 8, 0, 4, 5, 0, 2, 1, 2, 9,
        9, 4, 5, 4, 5, 9, 5, 0, 7, 1, 8, 0, 5, 7, 8, 1, 7, 7, 7, 2, 0, 3, 2, 5,
        4, 6, 4, 0, 8, 9, 8, 7, 2, 4, 1, 3, 5, 2, 5, 9, 5, 8, 3, 3, 6, 6, 7, 7,
        1, 0, 0, 7, 8, 1, 3, 8, 6, 1, 1, 6, 8, 7, 4, 1, 4, 2, 0, 1, 7, 4, 4, 5,
        4, 4, 5, 0, 2, 8, 1, 5, 2, 3, 3, 7, 7, 6, 0, 7, 5, 7, 2, 1, 0, 9, 3, 9,
        8, 6, 8, 3, 2, 5, 3, 3, 3, 1, 7, 2, 6, 1, 1, 5, 2, 5, 9, 2, 1, 5, 9, 4,
        3, 5, 4, 5, 2, 2, 2, 4, 5, 1, 9, 4, 3, 6, 1, 4, 4, 0, 8, 0, 9, 7, 7, 4,
        7, 7, 8, 2, 6, 9, 7, 9, 8, 7, 6, 5, 7, 6, 8, 6, 7, 2, 2, 3, 1, 5, 9, 9,
        3, 7, 4, 7, 3, 3, 0, 6, 3, 3, 4, 5, 7, 4, 1, 7, 7, 5, 0, 9, 5, 1, 7, 0,
        6, 6, 5, 3, 5, 9, 0, 1, 9, 4, 1, 6, 4, 1, 6, 0, 1, 7, 7, 3, 3, 7, 0, 5,
        7, 7

In [68]:
y_test_inversed_encoded = label_encoder.inverse_transform(y_test)
predicted_test_labels_inversed_encoded = label_encoder.inverse_transform(predicted_test_labels)

In [69]:
#model_path = os.path.join(current_path, "saved_model.pt")
torch.save(model.state_dict(), "C:/Users/lly/Desktop/CUHK/Year 4/Sem 1/AIST2010/Project/Singers/saved_model.pt")
print(f"Model saved as \n") #{model_path}

print("Model saved.")

Model saved as 

Model saved.


In [70]:
singer_accuracy = {}
for singer in label_encoder.classes_:
    singer_indices = np.where(y_test_inversed_encoded == singer)[0]
    singer_true_labels = y_test_inversed_encoded[singer_indices]
    singer_predicted_labels = predicted_test_labels_inversed_encoded[singer_indices]
    singer_acc = accuracy_score(singer_true_labels, singer_predicted_labels)
    singer_accuracy[singer] = singer_acc

In [71]:
# Sort the singers based on accuracy in descending order
sorted_singer_accuracy = sorted(singer_accuracy.items(), key=lambda x: x[1], reverse=True)

In [72]:
# Print the singers and their accuracies
print("Singer Accuracies:")
for singer, acc in sorted_singer_accuracy:
    print(f"{singer}: {acc:.2%}")

Singer Accuracies:
The Weeknd: 87.23%
Billie Eilish: 75.00%
Justin Bieber: 75.00%
Adele: 68.75%
Ed Sheeran: 68.33%
Lady Gaga: 63.49%
Ariana Grande: 63.46%
Taylor Swift: 63.04%
Beyonce: 61.36%
Dua Lipa: 47.92%


In [73]:
# preprocessing
def preprocess_test(audio_data):
    # Convert audio to mono
    audio_data = librosa.to_mono(audio_data)

    # Normalize audio
    audio_data = librosa.util.normalize(audio_data)

    return audio_data

In [115]:
# Read the audio file
audio_file = "C:/Users/lly/Desktop/CUHK/Year 4/Sem 1/AIST2010/Project/Singers/Adele/Hello_Adele.wav"  # Specify the path to your audio file

audio_data, sr = librosa.load(audio_file, sr=22050)  # Read the audio file

normalized_audio = preprocess_test(audio_data)

In [116]:
# Segment the audio into 1-second chunks
segment_duration = 1  # Duration of each segment in seconds
segment_length = int(segment_duration * sr)
num_segments = len(normalized_audio) // segment_length
segments = np.array_split(normalized_audio[:num_segments * segment_length], num_segments)

In [117]:
mfcc_features = []
for segment in segments:
    mfcc_segment = mfcc(segment)
    mfcc_features.append(mfcc_segment)

In [118]:
# Convert to numpy array
mfcc_features = np.array(mfcc_features)

# Standardize the MFCC features
scaler = StandardScaler()
mfcc_features = scaler.fit_transform(mfcc_features)

mfcc_tensor = torch.tensor(mfcc_features, dtype=torch.float32)

In [119]:
with torch.no_grad():
    outputs = model(mfcc_tensor.unsqueeze(1)) 
    probabilities = F.softmax(outputs, dim=1)
    #outputs = outputs.squeeze(1)
    #_, predicted_labels = torch.max(outputs, 1)
    predicted_labels = torch.argmax(probabilities, dim=1)
    predicted_singers = label_encoder.inverse_transform(predicted_labels.numpy())

In [120]:
probabilities

tensor([[2.1569e-01, 5.0080e-01, 1.6672e-02,  ..., 5.1729e-02, 4.9037e-03,
         2.8663e-05],
        [2.3798e-12, 2.2204e-07, 5.1935e-01,  ..., 2.8656e-07, 1.0202e-15,
         1.2227e-06],
        [2.6493e-07, 4.1107e-03, 4.8497e-02,  ..., 7.5780e-04, 2.2883e-04,
         4.0406e-06],
        ...,
        [1.9574e-10, 8.2881e-19, 2.6104e-12,  ..., 1.0332e-10, 2.5189e-06,
         1.4987e-14],
        [7.9626e-01, 8.5403e-02, 5.4124e-07,  ..., 1.1833e-01, 1.2302e-07,
         2.7325e-19],
        [1.1015e-01, 8.0866e-05, 7.0667e-05,  ..., 7.5428e-01, 1.2210e-02,
         1.1297e-11]])

In [121]:
highest_probability_index = torch.argmax(probabilities.mean(dim=0))
highest_probability_singer = label_encoder.classes_[highest_probability_index]

In [122]:
print("Predicted Singers:", predicted_singers)
print("Singer with Highest Probability:", highest_probability_singer)

Predicted Singers: ['Ariana Grande' 'Beyonce' 'Billie Eilish' 'Ariana Grande' 'Billie Eilish'
 'Beyonce' 'Ed Sheeran' 'Adele' 'Adele' 'Ed Sheeran' 'Ariana Grande'
 'Ed Sheeran' 'Beyonce' 'Beyonce' 'Beyonce' 'Ed Sheeran' 'Billie Eilish'
 'Beyonce' 'Beyonce' 'Ariana Grande' 'Lady Gaga' 'Lady Gaga'
 'Billie Eilish' 'Ed Sheeran' 'Beyonce' 'Adele' 'Ed Sheeran'
 'Ariana Grande' 'Ed Sheeran' 'Justin Bieber' 'Lady Gaga' 'Adele'
 'Lady Gaga' 'Adele' 'Ed Sheeran' 'Beyonce' 'Dua Lipa' 'Ed Sheeran'
 'Billie Eilish' 'Lady Gaga' 'Billie Eilish' 'Billie Eilish' 'Lady Gaga'
 'Lady Gaga' 'Ariana Grande' 'Dua Lipa' 'Ed Sheeran' 'Adele' 'Beyonce'
 'Ariana Grande' 'Ariana Grande' 'Dua Lipa' 'Ed Sheeran' 'The Weeknd'
 'Billie Eilish' 'Ed Sheeran' 'Lady Gaga' 'Lady Gaga' 'Ed Sheeran'
 'Beyonce' 'Billie Eilish' 'Beyonce' 'Taylor Swift' 'Adele' 'Beyonce'
 'Beyonce' 'Taylor Swift' 'Ariana Grande' 'Lady Gaga' 'Adele'
 'Taylor Swift' 'Beyonce' 'Taylor Swift' 'Taylor Swift' 'Lady Gaga'
 'Taylor Swift' 'Lady Gaga'